In [1]:
import networkx as nx
from tqdm.notebook import tqdm
import json
from typing import List
import os
import glob
import matplotlib.pyplot as plt
import sys

sys.path.append("../")
from src.utils import load_artificial, generate_orderings

seed = 123

In [2]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)


def save(G, label, fpath):
    data = nx.node_link_data(G)
    data["label"] = label
    with open(fpath, "w") as f:
        json.dump(data, f, cls=SetEncoder)


def load(fpath):
    data = json.load(open(fpath))
    label = data["label"]
    del data["label"]
    G = nx.node_link_graph(data)
    return G, label

In [3]:
def generate_random_regular_graphs(
    degrees: List,
    number_of_nodes,
    graphs_per_class,
    directory="../datasets/regular_graphs",
) -> None:
    """
    This function generates balanced dataset
    that consists of len(degrees)*number_of_graphs
    random regular graphs.

    Args:
        degrees: a list of degrees of nodes in respectful classes
        number_of_nodes: the number of graph vertices in all classes
        number_of_graphs: number of graphs in each class
        directory: output directory
    """
    graphs = []
    labels = []
    counter = 0

    if not os.path.exists(directory):
        os.makedirs(directory)

    for d, class_size in tqdm(zip(degrees, graphs_per_class)):
        for i in tqdm(range(class_size)):
            counter += 1
            G = nx.random_regular_graph(d=d, n=number_of_nodes, seed=seed)
            save(G=G, label=d, fpath=f"{directory}/{counter}.json")


def generate_partition_graphs(
    sizes: List[List],
    p_in: float,
    p_out: float,
    graphs_per_class: int,
    directory: str = "../datasets/partition_graphs",
) -> None:
    """
    This function generates balanced dataset
    that consists of len(degrees)*number_of_graphs
    random partition graphs.

    Args:
        sizes: sizes of groups
        p_in: probability of edges within groups
        p_out: probability of edges between groups
        number_of_graphs: number of graphs in each class
        directory: output directory
    """

    assert p_in >= 0 and p_in <= 1
    assert p_out >= 0 and p_out <= 1

    graphs = []
    labels = []
    counter = 0

    if not os.path.exists(directory):
        os.makedirs(directory)

    for s, class_size in tqdm(zip(sizes, graphs_per_class)):
        for i in tqdm(range(class_size)):
            counter += 1
            G = nx.random_partition_graph(s, p_in, p_out)
            save(G=G, label=s[0], fpath=f"{directory}/{counter}.json")

In [4]:
# Test 1.
generate_random_regular_graphs(
    degrees=[5, 7, 10], number_of_nodes=80, graphs_per_class=[105, 200, 170]
)

directory = "../datasets/regular_graphs"

graphs, labels = load_artificial(directory)

generate_orderings(directory, graphs)

# for fpath in tqdm(glob.glob(f"{directory}/*.json")):
#     G, label = load(fpath)
#     nx.draw(G)
#     plt.show()

0it [00:00, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

...

Calculating current_flow_closeness_centrality...


  0%|          | 0/475 [00:00<?, ?it/s]

<class 'networkx.utils.decorators.argmap'> compilation 20:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


Ordering wrt. current_flow_closeness_centrality...


  0%|          | 0/475 [00:00<?, ?it/s]

In [5]:
# Test 2.
communities = [[2] * 30, [10] * 6, [30] * 2]
directory = "../datasets/partition_graphs"

generate_partition_graphs(
    sizes=communities, p_in=0.75, p_out=0.25, graphs_per_class=[350, 100, 450]
)

graphs, labels = load_artificial(directory)

generate_orderings(directory, graphs)

# for fpath in tqdm(glob.glob(f"{directory}/*.json")):
#     G, label = load(fpath)
#     nx.draw(G)
#     plt.show()

0it [00:00, ?it/s]

  0%|          | 0/350 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

...

  0%|          | 0/900 [00:00<?, ?it/s]

Calculating current_flow_closeness_centrality...


  0%|          | 0/900 [00:00<?, ?it/s]

Ordering wrt. current_flow_closeness_centrality...


  0%|          | 0/900 [00:00<?, ?it/s]